In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


관련 패키지 설치

In [ ]:
!pip install ratsnlp

#문서 분류 모델 학습하기

모델 환경 설정

In [ ]:
import torch
from ratsnlp.nlpbook.classification import ClassificationTrainArguments
args = ClassificationTrainArguments(
    pretrained_model_name = 'beomi/kcbert-base', #pretrain 마친 언어 모델의 이름
    downstream_corpus_name = 'nsmc', #downstream 데이터의 이름
    downstream_model_dir = '/gdrive/MyDrive/nlpbook/checkpoints-doccls', #fine tuning된 모델의 checkpoint 저장될 위치
    batch_size=32 if torch.cuda.is_available() else 4, #GPU를 선택했다면 32, TPU라면 4라는 뜻. 코랩에서 TPU는 보통 8개 코어가 할당된다
    learning_rate=5e-5, #보폭
    max_seq_length=128, #128보다 큰건 자르고, 짧은건 PAD를 붙여준다
    epochs=3, #학습 에포크 수
    tpu_cores=0 if torch.cuda.is_available() else 8, #GPU를 선택했다면 0, TPU라면 8이라는 뜻
    seed=7 #random seed
)

In [ ]:
#랜덤 시드 고정
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


In [ ]:
#각종 로그를 출력하는 로거 설정
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='document-classification', downstream_corpus_name='nsmc', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/MyDrive/nlpbook/checkpoints-doccls', max_seq_length=128, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, cpu_workers=4, fp16=False, tpu_cores=0)
INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='document-classification', downstream_corpus_name='nsmc', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/MyDrive/nlpbook/checkpoints-doccls', max_seq_length=128, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_siz

말뭉치 내려받기

In [ ]:
from Korpora import Korpora
Korpora.fetch(
    corpus_name = args.downstream_corpus_name,
    root_dir = args.downstream_corpus_root_dir,
    force_download = True
)

[nsmc] download ratings_train.txt: 14.6MB [00:00, 79.4MB/s]                           
[nsmc] download ratings_test.txt: 4.90MB [00:00, 42.0MB/s]                            


토크나이저 준비하기

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case = False
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

학습데이터 구축

- 딥러닝 모델을 학습하려면 학습데이터를 배치(batch) 단위로 지속적으로 모델에 공급해 주어야 합니다. 파이토치(PyTorch)에서는 이 역할을 데이터 로더(DataLoader)가 수행
- NsmcCorpus는 CSV 파일 형식의 NSMC 데이터를 “문장(영화 리뷰) + 레이블(긍정, 부정)” 형태로 읽어들이는 역할
- ClassificationDataset는 각 instance를 포함하는 데이터셋 역할을 수행.
ClassificationDataset 클래스를 좀 더 자세히 살펴보겠습니다. 이 클래스는 NsmcCorpus와 토크나이저를 품고 있는데요. NsmcCorpus가 넘겨준 데이터(문장, 레이블)를 모델이 학습할 수 있는 형태로 가공합니다. 다시 말해 문장을 토큰화하고 이를 인덱스로 변환하는 한편, 레이블 역시 정수(integer)로 바꿔주는 역할을 한다.
- ClassificationDataset은 input_ids, attention_mask, token_type_ids의 길이가 모두 128인 이유는 토큰 기준 최대 길이(max_seq_length)를 args에서 128로 설정해 두었기 때문입니다.
- input_ids에 패딩 토큰([PAD])의 인덱스에 해당하는 0이 많이 붙어 있음을 확인할 수 있습니다. 분석 대상 문장의 토큰 길이가 max_seq_length보다 짧아서입니다. 이보다 긴 문장일 경우 128로 줄입니다.
- attention_mask는 해당 토큰이 패딩 토큰인지(0) 아닌지(1)를 나타내며 token_type_ids는 세그먼트(segment) 정보로 기본값은 모두 0으로 넣습니다. label은 정수로 변환
- sampler는 샘플링 방식을 정의합니다. 데이터 로더는 배치를 만들 때 ClassificationDataset이 들고 있는 전체 인스턴스 가운데 batch_size 갯수만큼을 비복원(replacement=False) 랜덤 추출한다. 학습용 데이터 로더와 달리 평가용 데이터 로더는 SequentialSampler를 사용한다. SequentialSampler는 batch_size만큼의 갯수만큼을 인스턴스 순서대로 추출하는 역할을 한다. 학습 때 배치 구성은 랜덤으로 하는 것이 좋으나 평가할 때는 평가용 데이터 전체를 사용하기 때문에 굳이 랜덤으로 구성할 이유가 없기 때문에 SequentialSampler를 쓴다.
- collate_fn은 이렇게 뽑힌 인스턴스를 배치로 만드는 역할을 하는 함수

학습 데이터셋 구축

In [ ]:
from ratsnlp.nlpbook.classification import NsmcCorpus, ClassificationDataset
corpus = NsmcCorpus()
train_dataset = ClassificationDataset(
    args = args,
    corpus = corpus,
    tokenizer = tokenizer,
    mode = 'train'
)

학습 데이터 로더 구축

In [ ]:
from torch.utils.data import DataLoader, RandomSampler
train_dataloader = DataLoader(
    train_dataset,
    batch_size = args.batch_size,
    sampler = RandomSampler(train_dataset, replacement=False),
    collate_fn = nlpbook.data_collator,
    drop_last = False,
    num_workers = args.cpu_workers
)

평가용 데이터셋 구축

In [ ]:
val_dataset = ClassificationDataset(
    args = args,
    corpus = corpus,
    tokenizer = tokenizer,
    mode = 'test'
)

평가용 데이터 로더 구축

In [ ]:
from torch.utils.data import SequentialSampler
val_dataloader = DataLoader(
    val_dataset,
    batch_size = args.batch_size,
    sampler = SequentialSampler(val_dataset),
    collate_fn = nlpbook.data_collator,
    drop_last = False,
    num_workers = args.cpu_workers
)

모델 초기화
- 프리트레인이 완료된 BERT 모델을 읽고, 문서 분류를 수행할 모델을 초기화
- BertForSequenceClassification은 프리트레인을 마친 BERT 모델 위에 문서 분류용 태스크 모듈이 덧붙여진 형태의 모델 클래스

In [ ]:
#pretrained 된 BERT 모델을 읽고, 문서 분류를 수행할 모델을 초기화
from transformers import BertConfig, BertForSequenceClassification
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=corpus.num_labels,
)
#BertForSequenceClassification은 pretrained 된 BERT 모델 위에 문서 분류용 태스크 모듈이 덧붙여진 형태의 모델 클래스
model = BertForSequenceClassification.from_pretrained(
        args.pretrained_model_name,
        config=pretrained_model_config,
)

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

모델 학습시키기

In [ ]:
#Task 정의
from ratsnlp.nlpbook.classification import ClassificationTask
task = ClassificationTask(model, args)

In [ ]:
#Trainer 정의
trainer = nlpbook.get_trainer(args)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
#학습 개시
trainer.fit(
    task,
    train_dataloaders = train_dataloader,
    val_dataloaders = val_dataloader,
)

INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/core/optimizer.py:380: RuntimeWarning: Found unsupported keys in the optimizer configuration: {'scheduler'}
  rank_zero_warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | BertForSequenceClassification | 108 M 
--------------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
435.680   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

#학습 마친 모델 실전 투입하기

인퍼런스 설정

In [ ]:
from ratsnlp.nlpbook.classification import ClassificationDeployArguments
args = ClassificationDeployArguments(
    pretrained_model_name = 'beomi/kcbert-base', #앞에서 사용한 pretrained_model_name (해당 모델은 허깅페이스 라이브러리에 등록 되어 있어야 한다)
    downstream_model_dir = '/gdrive/MyDrive/nlpbook/checkpoints-doccls', #앞에서 사용한 fine tuning한 모델의 checkpoint 저장 위치
    max_seq_length = 128 #토큰 기준 입력 문장 최대 길이 default는 128
)

downstream_model_checkpoint_fpath: /gdrive/MyDrive/nlpbook/checkpoints-doccls/epoch=1-val_loss=0.27.ckpt


토크나이저 로드

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case = False
)

체크포인트 로드

In [ ]:
import torch
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location = torch.device('cpu')
)

In [ ]:
#BERT 설정 로드
from transformers import BertConfig
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels = fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel()
)

#BERT 모델 초기화
from transformers import BertForSequenceClassification
model = BertForSequenceClassification(pretrained_model_config)

#체크포인트 주입
model.load_state_dict({k.replace('model.',''): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})

#평가 모드로 전환
model.eval()

인퍼런스 함수

In [ ]:
def inference_fn(sentence):
    #input_id, attention_masks, token_type_ids 만들기
    inputs = tokenizer(
        [sentence],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,
    )
    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()}) #inputs를 텐서로 바꿔서 모델 계산하기
        prob = outputs.logits.softmax(dim=1)
        positive_prob = round(prob[0][1].item(), 4) #긍정/부정 확률 소수점 4자리로 반올림
        negative_prob = round(prob[0][0].item(), 4) #긍정/부정 확률 소수점 4자리로 반올림
        pred = "긍정 (positive)" if torch.argmax(prob) == 1 else "부정 (negative)" #예측 확률의 최댓값 위치에 따라 pred 만들기
    return {
        'sentence': sentence,
        'prediction': pred,
        'positive_data': f"긍정 {positive_prob}",
        'negative_data': f"부정 {negative_prob}",
        'positive_width': f"{positive_prob * 100}%",
        'negative_width': f"{negative_prob * 100}%",
    }

In [ ]:
inference_fn('좋아요')

{'sentence': '좋아요',
 'prediction': '긍정 (positive)',
 'positive_data': '긍정 0.9863',
 'negative_data': '부정 0.0137',
 'positive_width': '98.63%',
 'negative_width': '1.37%'}

#웹서비스

웹서비스 만들기 준비
- ngrok은 코랩 로컬에서 실행 중인 웹서비스를 안전하게 외부에서 접근 가능하도록 해주는 도구입니다.
- ngrok을 실행하려면 회원가입 후 로그인을 한 뒤 이곳에 접속해 인증 토큰(authtoken)을 확인해야 합니다.
- 확인된 authtoken이 test111이라면 다음과 같이 실행합니다.

In [ ]:
!pwd

/content


In [ ]:
!mkdir /root/.ngrok2 && echo "authtoken:  2KTww7XOEkN663tDzp1fRnphA2E_5x8UAQKw4GAhq3Bzp9t4m" > /root/.ngrok2/ngrok.yml

mkdir: cannot create directory ‘/root/.ngrok2’: File exists


웹서비스 개시

In [ ]:
from ratsnlp.nlpbook.classification import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()